In [52]:
import numpy as np
import torch
import torch_geometric.datasets as datasets
import torch_geometric.data as data
import torch_geometric.transforms as transforms
import networkx as nx
from torch_geometric.utils.convert import to_networkx

In [53]:
import pandas as pd
df=pd.read_csv(r"C:\Users\tubaid\PycharmProjects\smiple_GNN\GraphNeuralNet-master\ITEMS.csv")

# df.head()
skills=df['SkillId'].tolist()
print(skills)

[451, 475, 477, 728, 967, 979, 981, 983, 445, 456, 462, 864, 940, 448, 469, 516, 527, 712, 716, 443, 453, 464, 468, 474, 446, 470, 941, 974, 975, 977, 520, 523, 525, 526, 714, 721, 519, 524, 962, 439, 450, 717, 971, 989, 454, 459, 460, 463, 467, 512, 432, 437, 457, 722, 865, 869, 438, 440, 452, 713, 969, 461, 466, 482, 486, 472, 866, 984, 444, 484, 485, 488, 517, 723, 441, 442, 515, 870, 965, 968, 985, 2005, 433, 434, 455, 476, 478, 514, 522, 720, 990, 431, 513, 518, 727, 867, 2011, 2012, 961, 973, 980, 2024, 2026, 471, 479, 483, 521, 719, 3687, 868, 964, 943, 992, 2025, 2030, 2006, 944, 982, 991, 2007, 963, 966, 2023, 2008, 2019, 2028, 970, 986, 988, 2015, 2017, 2009, 2014, 715, 726, 960, 2010, 2013, 2016, 2020, 2021, 2022, 976, 3686, 435, 449, 473, 480, 481, 718, 972, 978, 436, 447, 465, 487, 724, 725, 942, 987, 2018, 2027, 2029]


In [54]:
df=df.drop( columns=['SkillId'],axis=1)
features = df.to_numpy().tolist()
# print(features)

In [55]:
x = torch.tensor(features, dtype=torch.float)
y = torch.tensor(skills, dtype=torch.float)
print(x.shape)
print(y.shape)
print(len(x))
print(len(y))

torch.Size([163, 10])
torch.Size([163])
163
163


In [56]:
preReq=pd.read_excel(r"C:\Users\tubaid\PycharmProjects\smiple_GNN\GraphNeuralNet-master\skils.xlsx")
ls=preReq.SkillId.unique().tolist()
# print('total',len(preReq.SkillId.tolist()))
# print('unique',ls)
print(len(ls))

skID = list(preReq['SkillId'])
a_string = preReq['Prereqs'].fillna('0')

pReq = []
tmp = []
for i in range(len(a_string)):
    a_string[i] = a_string[i].replace(",", " ")
#     print(f'String {i} = {a_string[i]}')
    if len(a_string[i])>6:
        for word in a_string[i].split():
            if word.isdigit():
                tmp.append(int(word))
        pReq.append(tmp)
        tmp=[]
    else:
        for word in a_string[i].split():
            if word.isdigit():
                pReq.append([int(word)])

gS_Data = []

# print(gS_Data)
gPR_Data = []
for i in range(len(skID)):
    for j in range (len(pReq[i])):
        gS_Data.append(skID[i])
        gPR_Data.append(pReq[i][j])

gDf = pd.DataFrame({
    'SkillId':gS_Data,
    'PreRequisits':gPR_Data
})

579


In [57]:
import torch
from torch_geometric.data import Data
edge_index = torch.tensor([gPR_Data,gS_Data], dtype=torch.long)
dt=Data(x=x, y=y, edge_index=edge_index)

In [58]:
dt=Data(x=x, y=y, edge_index=edge_index)
dt

Data(x=[163, 10], edge_index=[2, 883], y=[163])

In [59]:
x.shape

torch.Size([163, 10])

In [60]:
from torch_geometric.datasets import Planetoid
import torch
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

In [61]:
class GCNConv(MessagePassing):
    # Applies a linear tranformation to the incoming data
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr='add')  # "Add" aggregation
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # Step 1: Add self-loops
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Multiply with weights
        x = self.lin(x)

        # Step 3: Calculate the normalization
        row, col = edge_index
        deg = degree(row, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # Step 4: Propagate the embeddings to the next layer
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x,
                              norm=norm)

    def message(self, x_j, norm):
        # Normalize node features.
        return norm.view(-1, 1) * x_j


In [62]:
class Net(torch.nn.Module):

    def __init__(self, dataset):
        super(Net, self).__init__()
        # two convolution layers of GCN
        # first represnt the input and the second represnt the output layer
        # dataset.num_node_features== number of input
        # 16 hidden layers
        # dataset.num_classes==outputclass
        self.conv1 = GCNConv(10, 16)
        self.conv2 = GCNConv(16, 7)
    # represent the weights updates
    # weights in output
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)



In [63]:
def plot_dataset(dataset):
    edges_raw = dataset.data.edge_index.numpy()
    edges = [(x, y) for x, y in zip(edges_raw[0, :], edges_raw[1, :])]
    labels = dataset.data.y.numpy()

    G = nx.Graph()
    G.add_nodes_from(list(range(np.max(edges_raw))))
    G.add_edges_from(edges)
    plt.subplot(111)
    options = {
                'node_size': 30,
                'width': 0.2,
    }
    nx.draw(G, with_labels=False, node_color=labels.tolist(), cmap=plt.cm.tab10, font_weight='bold', **options)
    plt.show()


In [64]:
def test(data, train=True):
    model.eval()

    correct = 0
    pred = model(data).max(dim=1)[1]

    if train:
        correct += pred[data.train_mask].eq(data.y[data.train_mask]).sum().item()
        return correct / (len(data.y[data.train_mask]))
    else:
        correct += pred[data.test_mask].eq(data.y[data.test_mask]).sum().item()
        return correct / (len(data.y[data.test_mask]))

In [65]:
def train(data, plot=False):
#     train_accuracies, test_accuracies = list(), list()
    train_accuracies, test_accuracies = [], []

    for epoch in range(100):
            model.train()
            optimizer.zero_grad()
            out = model(data)
            loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            train_acc = test(data)
            test_acc = test(data, train=False)

            train_accuracies.append(train_acc)
            test_accuracies.append(test_acc)
            print('Epoch: {:03d}, Loss: {:.5f}, Train Acc: {:.5f}, Test Acc: {:.5f}'.
                  format(epoch, loss, train_acc, test_acc))

    # torch.save(model.state_dict(), 'new_model_weights.pth')

In [66]:
# if plot:
#  plt.plot(train_accuracies, label="Train accuracy")
#  plt.plot(test_accuracies, label="Validation accuracy")
#  plt.xlabel("# Epoch")
#  plt.ylabel("Accuracy")
#  plt.legend(loc='upper right')
#  plt.show()

In [67]:
for i in dt:
    print(i)

('x', tensor([[5.8258e+01, 2.4926e+01, 1.0000e+00,  ..., 9.0000e+00, 2.4938e+03,
         8.1765e+00],
        [5.2378e+01, 1.7044e+01, 1.0000e+00,  ..., 4.6000e+01, 1.2573e+03,
         3.9291e+00],
        [4.6099e+01, 1.7424e+01, 1.0000e+00,  ..., 3.3000e+01, 1.2540e+03,
         4.3092e+00],
        ...,
        [3.4300e+01, 1.1673e+01, 0.0000e+00,  ..., 1.0000e+01, 3.9574e+02,
         5.1394e+00],
        [3.2677e+01, 1.2090e+01, 0.0000e+00,  ..., 1.2000e+01, 5.7344e+02,
         7.3518e+00],
        [2.7355e+01, 1.6197e+01, 0.0000e+00,  ..., 9.0000e+00, 2.3742e+02,
         4.3966e+00]]))
('edge_index', tensor([[    0,  3530,  3531,  ...,  8811, 10514,     0],
        [ 3530,  3531,  3532,  ..., 10432,  8808, 10549]]))
('y', tensor([ 451.,  475.,  477.,  728.,  967.,  979.,  981.,  983.,  445.,  456.,
         462.,  864.,  940.,  448.,  469.,  516.,  527.,  712.,  716.,  443.,
         453.,  464.,  468.,  474.,  446.,  470.,  941.,  974.,  975.,  977.,
         520.,  523.,  5

In [68]:
len(dt.y.tolist())

163

In [69]:
model = Net(dt)
# data = dataset[0].to(device)

In [70]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [71]:
train(dt,False)

RuntimeError: index 3530 is out of bounds for dimension 0 with size 163

In [ ]:
len(dt.x)